In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
split_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - split_window_size, split_window_size + 1):
    x_train_temp = x_data.iloc[i:i+split_window_size+1]
    y_train_temp = y_data.iloc[i:i+split_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [33]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [34]:
import torch
import torch.nn as nn
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [8]
num_layers_list = [2]
learning_rates = [ 0.001]
window_sizes = [1]
num_epochs = 500

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Function to create and train LSTM model with given hyperparameters
def train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor):
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

    return model

# Function to evaluate the model on test data
def evaluate_model(model, x_test_tensor, y_test_tensor, window_size):
    test_loss = 0.0
    model.eval()

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    return test_loss / len(x_test_tensor)

best_hyperparameters = None
best_test_loss = float('inf')

# Iterate over hyperparameter combinations and train models
for input_size, hidden_size, num_layers, learning_rate, window_size in hyperparameter_combinations:
    print(f"Training with hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    model = train_lstm_model(input_size, hidden_size, num_layers, learning_rate, window_size, x_train_tensor, y_train_tensor, x_val_tensor, y_val_tensor)
    test_loss = evaluate_model(model, x_test_tensor, y_test_tensor, window_size)

    print(f"Test Loss: {test_loss}")

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_hyperparameters = (input_size, hidden_size, num_layers, learning_rate, window_size)

print("Best Hyperparameters:")
print(f"input_size={best_hyperparameters[0]}, hidden_size={best_hyperparameters[1]}, num_layers={best_hyperparameters[2]}, learning_rate={best_hyperparameters[3]}, window_size={best_hyperparameters[4]}")


Training with hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.001, window_size=1


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/500], Training Loss: 20887.835165217624, Validation Loss: 21235.169921875
Epoch [2/500], Training Loss: 16789.660814101077, Validation Loss: 18935.70703125
Epoch [3/500], Training Loss: 13477.84038354132, Validation Loss: 14569.8212890625
Epoch [4/500], Training Loss: 11228.910932389292, Validation Loss: 11913.177734375
Epoch [5/500], Training Loss: 9570.47167468871, Validation Loss: 10585.619140625
Epoch [6/500], Training Loss: 8354.538875011864, Validation Loss: 9552.6533203125
Epoch [7/500], Training Loss: 7186.938264394703, Validation Loss: 11025.0478515625
Epoch [8/500], Training Loss: 6201.4360563806185, Validation Loss: 8171.79443359375
Epoch [9/500], Training Loss: 5309.7475636645, Validation Loss: 6545.21044921875
Epoch [10/500], Training Loss: 4581.155635527036, Validation Loss: 5483.7919921875
Epoch [11/500], Training Loss: 3955.173507548765, Validation Loss: 4715.3857421875
Epoch [12/500], Training Loss: 3400.622610118315, Validation Loss: 4068.77978515625
Epoch [1

In [35]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 8
num_layers = 2

learning_rate = 0.001

num_epochs = 300

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# Calculate test loss after training
test_loss = 0.0
model.eval()
with torch.no_grad():
    for i in range(len(x_test_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_test_tensor))

        inputs = x_test_tensor[i:window_end].unsqueeze(0)
        labels = y_test_tensor[window_end - 1]

        outputs = model(inputs)
        test_loss += criterion(outputs, labels)

print(f'Test Loss: {test_loss / len(x_test_tensor)}')


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300], Training Loss: 20850.550288373997, Validation Loss: 21213.60546875
Epoch [2/300], Training Loss: 16831.715232942737, Validation Loss: 18986.125
Epoch [3/300], Training Loss: 13502.97734956084, Validation Loss: 14966.30078125
Epoch [4/300], Training Loss: 11241.155068569682, Validation Loss: 11896.580078125
Epoch [5/300], Training Loss: 9573.374792077298, Validation Loss: 10598.3017578125
Epoch [6/300], Training Loss: 14802.57295829931, Validation Loss: 11284.69921875
Epoch [7/300], Training Loss: 8190.930847354726, Validation Loss: 9639.861328125
Epoch [8/300], Training Loss: 7860.064874516047, Validation Loss: 12760.9892578125
Epoch [9/300], Training Loss: 6255.211520377863, Validation Loss: 12711.2607421875
Epoch [10/300], Training Loss: 4830.505320496257, Validation Loss: 7154.4970703125
Epoch [11/300], Training Loss: 4132.376038949922, Validation Loss: 5312.2060546875
Epoch [12/300], Training Loss: 3547.0474516417307, Validation Loss: 4350.71826171875
Epoch [13/300],

In [ ]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 8
num_layers = 3

learning_rate = 0.001

num_epochs = 300

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# Calculate test loss after training
test_loss = 0.0
model.eval()
with torch.no_grad():
    for i in range(len(x_test_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_test_tensor))

        inputs = x_test_tensor[i:window_end].unsqueeze(0)
        labels = y_test_tensor[window_end - 1]

        outputs = model(inputs)
        test_loss += criterion(outputs, labels)

print(f'Test Loss: {test_loss / len(x_test_tensor)}')


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300], Training Loss: 20890.032764768628, Validation Loss: 21236.92578125
Epoch [2/300], Training Loss: 17500.138662843186, Validation Loss: 19276.255859375
Epoch [3/300], Training Loss: 13969.773321590807, Validation Loss: 16890.3203125
Epoch [4/300], Training Loss: 11619.360474948111, Validation Loss: 14103.7861328125
Epoch [5/300], Training Loss: 9903.484623459302, Validation Loss: 11468.2314453125
Epoch [6/300], Training Loss: 8551.398639761945, Validation Loss: 11284.8984375
Epoch [7/300], Training Loss: 7397.702266702298, Validation Loss: 9553.578125
Epoch [8/300], Training Loss: 6416.287938114935, Validation Loss: 7842.08154296875
Epoch [9/300], Training Loss: 5547.303140038055, Validation Loss: 6574.96240234375
Epoch [10/300], Training Loss: 4813.158028178025, Validation Loss: 5700.2568359375
Epoch [11/300], Training Loss: 4182.2834019868615, Validation Loss: 4944.38330078125
Epoch [12/300], Training Loss: 3614.4943341520784, Validation Loss: 4247.64013671875
Epoch [13/

KeyboardInterrupt: 

In [ ]:
ö

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 2

learning_rate = 0.001

num_epochs = 300

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# Calculate test loss after training
test_loss = 0.0
model.eval()
with torch.no_grad():
    for i in range(len(x_test_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_test_tensor))

        inputs = x_test_tensor[i:window_end].unsqueeze(0)
        labels = y_test_tensor[window_end - 1]

        outputs = model(inputs)
        test_loss += criterion(outputs, labels)

print(f'Test Loss: {test_loss / len(x_test_tensor)}')


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300], Training Loss: 17321.558742325018, Validation Loss: 18885.462890625
Epoch [2/300], Training Loss: 11926.567034546179, Validation Loss: 13731.6337890625
Epoch [3/300], Training Loss: 8786.351654449358, Validation Loss: 11032.8232421875
Epoch [4/300], Training Loss: 6675.842657987934, Validation Loss: 8370.7119140625
Epoch [5/300], Training Loss: 5050.248162783188, Validation Loss: 5932.76708984375
Epoch [6/300], Training Loss: 3850.393570966616, Validation Loss: 4313.80517578125
Epoch [7/300], Training Loss: 2899.5824525393873, Validation Loss: 3221.017333984375
Epoch [8/300], Training Loss: 2207.1526585015986, Validation Loss: 2448.67578125
Epoch [9/300], Training Loss: 1714.35814106997, Validation Loss: 1920.7452392578125
Epoch [10/300], Training Loss: 1346.6013982689726, Validation Loss: 1543.0526123046875
Epoch [11/300], Training Loss: 1077.4193558688632, Validation Loss: 1279.5889892578125
Epoch [12/300], Training Loss: 864.9449481852663, Validation Loss: 1062.318115

KeyboardInterrupt: 

In [ ]:
ä

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 3

learning_rate = 0.001

num_epochs = 300

model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# Calculate test loss after training
test_loss = 0.0
model.eval()
with torch.no_grad():
    for i in range(len(x_test_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_test_tensor))

        inputs = x_test_tensor[i:window_end].unsqueeze(0)
        labels = y_test_tensor[window_end - 1]

        outputs = model(inputs)
        test_loss += criterion(outputs, labels)

print(f'Test Loss: {test_loss / len(x_test_tensor)}')


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300], Training Loss: 17677.66852652012, Validation Loss: 18983.8828125
Epoch [2/300], Training Loss: 13314.560460591827, Validation Loss: 19043.494140625
Epoch [3/300], Training Loss: 9415.50923587024, Validation Loss: 13117.5380859375
Epoch [4/300], Training Loss: 7162.658139457555, Validation Loss: 11429.8466796875
Epoch [5/300], Training Loss: 5448.702984772235, Validation Loss: 8781.126953125
Epoch [6/300], Training Loss: 4167.971235568658, Validation Loss: 5319.03271484375
Epoch [7/300], Training Loss: 3147.9685416147195, Validation Loss: 3695.0166015625
Epoch [8/300], Training Loss: 2385.342645815755, Validation Loss: 2712.600341796875
Epoch [9/300], Training Loss: 1852.4095450932532, Validation Loss: 2096.551025390625
Epoch [10/300], Training Loss: 1446.0212057942897, Validation Loss: 1649.9329833984375
Epoch [11/300], Training Loss: 1161.8481102767985, Validation Loss: 1360.052001953125
Epoch [12/300], Training Loss: 931.3764763119127, Validation Loss: 1122.69213867187

In [ ]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 3

learning_rate = 0.001



num_epochs = 75


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/75], Training Loss: 17670.56007460006, Validation Loss: 18982.166015625
Epoch [2/75], Training Loss: 13317.910812193204, Validation Loss: 19067.453125
Epoch [3/75], Training Loss: 9439.795367973113, Validation Loss: 12179.8896484375
Epoch [4/75], Training Loss: 7184.406515963105, Validation Loss: 9667.3212890625
Epoch [5/75], Training Loss: 5462.750215013006, Validation Loss: 7110.85986328125
Epoch [6/75], Training Loss: 4184.5808915649195, Validation Loss: 5073.791015625
Epoch [7/75], Training Loss: 3162.2398899728682, Validation Loss: 3616.109619140625
Epoch [8/75], Training Loss: 2396.2662249388904, Validation Loss: 2747.4453125
Epoch [9/75], Training Loss: 1865.7307931666057, Validation Loss: 2163.51708984375
Epoch [10/75], Training Loss: 1457.1492026851538, Validation Loss: 1701.108154296875
Epoch [11/75], Training Loss: 1169.6494664359911, Validation Loss: 1412.09716796875
Epoch [12/75], Training Loss: 938.6326561313589, Validation Loss: 1139.8902587890625
Epoch [13/75],

In [ ]:
ö

NameError: name 'ö' is not defined

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 4

learning_rate = 0.001



num_epochs = 100


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Training Loss: 17730.7561042866, Validation Loss: 18999.392578125
Epoch [2/100], Training Loss: 14282.744352764164, Validation Loss: 18811.552734375
Epoch [3/100], Training Loss: 10679.519531336262, Validation Loss: 21464.904296875
Epoch [4/100], Training Loss: 8058.161620794091, Validation Loss: 14014.69921875
Epoch [5/100], Training Loss: 6092.340851367106, Validation Loss: 10865.939453125
Epoch [6/100], Training Loss: 4633.005494101313, Validation Loss: 7484.09619140625
Epoch [7/100], Training Loss: 3535.1883968077527, Validation Loss: 5420.81201171875
Epoch [8/100], Training Loss: 2684.151329534713, Validation Loss: 4247.88330078125
Epoch [9/100], Training Loss: 2095.565768319551, Validation Loss: 3262.3642578125
Epoch [10/100], Training Loss: 1660.5040386335654, Validation Loss: 2421.912109375
Epoch [11/100], Training Loss: 1348.481363661686, Validation Loss: 1877.48583984375
Epoch [12/100], Training Loss: 1116.5901727804567, Validation Loss: 1623.978515625
Epoch [1

KeyboardInterrupt: 

In [ ]:
öö

In [ ]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 4

learning_rate = 0.001



num_epochs = 50


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 17720.693621782913, Validation Loss: 18996.841796875
Epoch [2/50], Training Loss: 14272.882685991912, Validation Loss: 18814.1875
Epoch [3/50], Training Loss: 10674.680861884675, Validation Loss: 21226.615234375
Epoch [4/50], Training Loss: 8588.614925458753, Validation Loss: 15010.2470703125
Epoch [5/50], Training Loss: 6185.099233565122, Validation Loss: 11488.2373046875
Epoch [6/50], Training Loss: 4684.1439117069285, Validation Loss: 8399.1201171875
Epoch [7/50], Training Loss: 3575.1960553159265, Validation Loss: 5951.4814453125
Epoch [8/50], Training Loss: 2705.8135135599478, Validation Loss: 3569.144775390625
Epoch [9/50], Training Loss: 2090.6786706377957, Validation Loss: 2486.810302734375
Epoch [10/50], Training Loss: 1637.921615724722, Validation Loss: 1955.5142822265625
Epoch [11/50], Training Loss: 1318.6214037300406, Validation Loss: 1642.831298828125
Epoch [12/50], Training Loss: 1073.980153752395, Validation Loss: 1423.8170166015625
Epoch [1